In [1]:
from datasets import load_dataset


/data/katie_kang/miniconda3/envs/trlx2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
dataset = load_dataset("fever/fever", 'v1.0')

Generating paper_test split: 100%|██████████| 18567/18567 [00:00<00:00, 30423.10 examples/s]


In [62]:
dataset["train"]

Dataset({
    features: ['id', 'label', 'claim', 'evidence_annotation_id', 'evidence_id', 'evidence_wiki_url', 'evidence_sentence_id'],
    num_rows: 311431
})

In [74]:
dataset["train"][5]

{'id': 156709,
 'label': 'REFUTES',
 'claim': 'Adrienne Bailon is an accountant.',
 'evidence_annotation_id': 180804,
 'evidence_id': 193183,
 'evidence_wiki_url': 'Adrienne_Bailon',
 'evidence_sentence_id': -1}

In [2]:

dataset = load_dataset("hotpotqa/hotpot_qa","distractor")

/data/katie_kang/miniconda3/envs/trlx2/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for hotpotqa/hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpotqa/hotpot_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
dataset["train"]

Dataset({
    features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
    num_rows: 90447
})

In [17]:
dataset["train"][10][]

{'id': '5abd90545542996e802b47d7',
 'question': 'Fast Cars, Danger, Fire and Knives includes guest appearances from which hip hop record executive?',
 'answer': 'Jaime Meline',
 'type': 'bridge',
 'level': 'medium',
 'supporting_facts': {'title': ['Fast Cars, Danger, Fire and Knives', 'El-P'],
  'sent_id': [2, 0]},
 'context': {'title': ['Lights Out Paris',
   'El-P',
   'Born and Raised (EP)',
   'Lord Steppington',
   'Control Freek',
   'Hip hop',
   'Fast Cars, Danger, Fire and Knives',
   'Longterm Mentality',
   'Experimental hip hop',
   'Changes (Alyson Avenue album)'],
  'sentences': [['Lights Out Paris is the first studio album by American hip hop artist Sims, a member of Minneapolis indie hip hop collective Doomtree.',
    ' It was released July 28, 2005 on Doomtree Records and includes guest appearances from P.O.S, Crescent Moon, and Toki Wright, among others.',
    ' The album was re-released with four remixes and five songs from Sims\' "False Hopes Four" on vinyl in June 

In [ ]:
problem_template = (
"""Answer the following question using the given facts
Facts: {facts_subsampled}
Question: {question}"""
)

initial = (
"""## Instruction:
Answer the following question to be best of your ability

{question}
## Answer:
"""
)

decide_ask_0 = (
"""{context}

## Instruction:
Reflect on your solution and what you know now. Are there any remaining parts you're unsure about? Answer "yes" or "no" (one word only on its own line, nothing else).
## Are you still unsure? [yes/no]:"""
)


decide_ask_1 = (
"""{context}

## Instruction:
Reflect on your solution and what you know now. Are there any remaining parts you're unsure about? (hint: your solution is probably wrong). Answer "yes" or "no" (one word only on its own line, nothing else).
## Are you still unsure? [yes/no]:"""
)

question = (
"""{context}

## Instruction:
Explain what you're unsure about and ask for help. State as explicitly as possible where you are missing information to answer the question.
## Your question:"""
)

final = (
"""{context}

## Instruction:
Output your final solution.

## Answer:
"""
)

final_with_ask = (
"""{context}

## Instruction:
Based on the information you received, make any necessary changes to your solution.

First, explain about what you were incorrect about and what changes you will make to your answer given the new information. Then, output the correct answer.

Start your final answer with "Answer: "

## Your final solution:
Reasoning:"""
)


teacher_sys = (
"""You are a helpful teacher helping a student with their homework. The student is asking you questions about their homework.

Your are provided with a list of facts to help you answer the student's questions. 
If the student's question is outside the scope of the provided facts, answer "I don't know".
Otherwise, answer the student's question to the best of your ability using the provided facts. Be specific and succinct in your answer.

"""
)
teacher_msg = (
"""## Facts: {facts_full}

## Student question: {question}
## Answer:"""
)


CODE_PROMPTS = {
  "problem_template": problem_template.strip(),
  "initial": initial.strip(),
  "decide_ask_0": decide_ask_0.strip(),
  "decide_ask_1": decide_ask_1.strip(),
  "question": question.strip(),
  "final": final.strip(),
  "final_with_ask": final_with_ask.strip(),
  "teacher_sys": teacher_sys.strip(),
  "teacher_msg": teacher_msg.strip(),
}


: 

In [4]:
dataset["train"][10].keys()

dict_keys(['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'])

In [16]:
dataset["train"][20]["supporting_facts"]

{'title': ['Warren Bryant', 'Longs Drugs'], 'sent_id': [0, 0]}

In [18]:
dataset["train"][10]["context"]["sentences"]

[['Lights Out Paris is the first studio album by American hip hop artist Sims, a member of Minneapolis indie hip hop collective Doomtree.',
  ' It was released July 28, 2005 on Doomtree Records and includes guest appearances from P.O.S, Crescent Moon, and Toki Wright, among others.',
  ' The album was re-released with four remixes and five songs from Sims\' "False Hopes Four" on vinyl in June 2015.'],
 ['Jaime Meline (born March 2, 1975), better known by his stage name El-P (shortened from El Producto), is an American hip hop recording artist, record producer, and record executive.',
  ' Originally a member of Company Flow, El-P has been a major driving force in alternative hip hop for more than two decades, producing for several notable rappers such as Aesop Rock, Mr. Lif, and Cage, among others.'],
 ['Born and Raised is the debut EP by American hip hop duo Smif-N-Wessun, released on December 3, 2013, under Duck Down Music Inc..',
  ' Entirely produced by Beatnick & K-Salaam, the 6-so

In [38]:
import numpy as np 
def get_supporting_facts(example):
    supporting_facts_info = example["supporting_facts"]
    supporting_facts_all = example["context"]
    
    titles = []
    supporting_facts = []
    for fact_idx in range(len(supporting_facts_info["title"])):
        title = supporting_facts_info["title"][fact_idx]
        context_idx = np.where(np.array(supporting_facts_all["title"]) == title)[0][0]
        fact = supporting_facts_all["sentences"][context_idx][supporting_facts_info["sent_id"][fact_idx]]
        titles.append(title)
        supporting_facts.append(fact)
    return titles, supporting_facts
        

In [55]:
idx = 150
# idx = 152
print(dataset["train"][idx]["question"])
print(dataset["train"][idx]["answer"])
titles, supporting_facts = (get_supporting_facts(dataset["train"][idx]))

for i in range(len(titles)):
    print(titles[i]+": "+ supporting_facts[i])

Abraham Weinberg was a suspect in the killing of which Irish-American hitman?
Vincent "Mad Dog" Coll
Abraham Weinberg:  Abe Weinberg would become one of Schultz's top gunmen during the Manhattan Bootleg Wars and was a later suspect in the high-profile gangland slayings of Jack "Legs" Diamond, Vincent "Mad Dog" Coll, and mob boss Salvatore Maranzano.
Mad Dog Coll: Vincent "Mad Dog" Coll (born Uinseann Ó Colla, July 20, 1908 – February 7, 1932) was an Irish American mob hitman in the 1920s and early 1930s in New York City.


In [52]:
idx = 153
print(dataset["train"][idx]["question"])
print(dataset["train"][idx]["answer"])
titles, supporting_facts = (get_supporting_facts(dataset["train"][idx]))

for i in range(len(titles)):
    print(titles[i]+": "+ supporting_facts[i])

What was population of the city where the Dutch retired footballer Foeke Booy born?
108,249
Foeke Booy: Foeke Booy (born 25 April 1962 in Leeuwarden) is a Dutch retired footballer and football manager.
Leeuwarden: Leeuwarden (] , ] ), Stadsfries: "Liwwadden") is a city and municipality with a population of 108,249 in Friesland in the Netherlands.


In [58]:
idx = 158
print(dataset["train"][idx]["question"])
print(dataset["train"][idx]["answer"])
titles, supporting_facts = (get_supporting_facts(dataset["train"][idx]))

for i in range(len(titles)):
    print(titles[i]+": "+ supporting_facts[i])

Mackenzie Davis appeared in the 2013 Canadian romantic comedy film directed by whom?
Michael Dowse
Mackenzie Davis:  She made her film debut in "Smashed", and later appeared in "Breathe In", "That Awkward Moment", and "The F Word", the latter for which she received a Canadian Screen Award nomination.
The F Word (2013 film):  is a 2013 Canadian romantic comedy film directed by Michael Dowse and written by Elan Mastai, based on TJ Dawe and Michael Rinaldi's play "Toothpaste and Cigars".


In [ ]:
# one fact gives all the info away

# The 2000–01 NBA season was the Jazz's 27th season in the National Basketball Association, and 22nd season in Salt Lake City, Utah, after the retirement of Jeff Hornacek, the Jazz signed free agents John Starks, and who?
# Danny Manning
# 2000–01 Utah Jazz season:  After the retirement of Jeff Hornacek, the Jazz signed free agents Danny Manning and John Starks, and acquired Donyell Marshall from the Golden State Warriors in an offseason four-team trade.
# John Starks (basketball): John Levell Starks (born August 10, 1965) is an American retired professional basketball shooting guard.

In [ ]:
# you can guess the answer from just the presense of some facts

# Who chaired the ceremony in which the film written by Canet and Philippe Lefebvre won the award for Best Film?
# Isabelle Mergault
# 12th Lumières Awards:  The ceremony was chaired by Isabelle Mergault. "
# 12th Lumières Awards: Tell No One" won the award for Best Film.
# Tell No One:  Written by Canet and Philippe Lefebvre and starring François Cluzet, the film won four categories at the 2007 César Awards in France: Best Director (Guillaume Canet), Best Actor (François Cluzet), Best Editing and Best Music Written for a Film.


# Given first 2 facts, GPT answers 
# Who chaired the 12th Lumières Awards ceremony in which "Tell No One" won the award for Best Film?


In [ ]:
# GPT just rephrases the question

# There is some information missing from the provided fact in order to answer the given question. Ask a question to query for the missing information. The question to query for the least amount of information necessary.

# Question: Abraham Weinberg was a suspect in the killing of which Irish-American hitman?

# Facts:
# Abraham Weinberg: Abe Weinberg would become one of Schultz's top gunmen during the Manhattan Bootleg Wars and was a later suspect in the high-profile gangland slayings of Jack "Legs" Diamond, Vincent "Mad Dog" Coll, and mob boss Salvatore Maranzano.
# Mad Dog Coll: Vincent "Mad Dog" Coll (born Uinseann Ó Colla, July 20, 1908 – February 7, 1932) was an Irish American mob hitman in the 1920s and early 1930s in New York City.

# Given fact 1
# Which Irish-American hitman was among the high-profile gangland slayings in which Abraham Weinberg was a suspect?







In [ ]:

questions = []
answers = []
facts = []



    texts = [
      self.prompts["problem_template"].format(
        task=p["text"],
        tests="\n".join(p["test_list"]),
      )
      for p in self.problems
    ]
    contexts = [
      {
        "problem": p,
        "text": t,
        "done": False,
        "invalid": False,
        "correct": False
      } for p, t in zip(self.problems, texts)
    ]

    # Generate initial solution
    contexts = self.initial_answer(contexts)
    invalid = [c for c in contexts if c["invalid"]]
    contexts = [c for c in contexts if not c["invalid"]]
    print(
      f" => Generating initial: {len(contexts)} pass | {len(invalid)} invalid"
      f" | {len(self.problems)} total"
    )

    # Decide
    contexts = self.decide_ask(contexts)
    invalid += [c for c in contexts if c["invalid"]]
    contexts = [c for c in contexts if not c["invalid"]]

    # Ask for the rest
    complete = [c for c in contexts if c["done"]]
    incomplete = [c for c in contexts if not c["done"]]
    incomplete_and_wrong = [c for c in incomplete if not c["initial_correct"]]
    print(
      f" => After decide ask: {len(contexts)} pass"
      f" : {len(complete)} done (no ask) {len(incomplete)} want to ask {len(incomplete_and_wrong)} actually ask"
      f" | {len(invalid)} invalid"
    )
    rest = self.ask_questions(incomplete_and_wrong)
    contexts = complete + incomplete + rest
    print(
      f" => After ask branching: {len(contexts)} pass"
      f" : {len(incomplete_and_wrong)} asked -> {len(rest)} branched"
      f" | {len(invalid)} invalid"
    )

    # Final
    contexts = self.final_answer(contexts)
    invalid += [c for c in contexts if c["invalid"]]
    contexts = [c for c in contexts if not c["invalid"]]
    print(
      f" => After final: {len(contexts)} pass"
      f" | {len(invalid)} invalid"
    )

    contexts = self.evaluate_improvement(contexts)
    dataset = [c for c in contexts if not c["invalid"] and c["evaluator_better"]]
    print(
      f" => After evaluator: {len(dataset)} dataset"
      f" : {len([c for c in contexts if c['evaluator_better']])} better"
      f" | {len(invalid)} invalid"
    )